<a href="https://colab.research.google.com/github/Aksinhaa/ColabFold/blob/main/Genomics_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<video src="https://raw.githubusercontent.com/PoODL-CES/PoODL_NGS_workshop.io/main/Nature%20of%20Science.mp4" controls autoplay loop width="400" align="right"></video>

## Genomics Learning Workshop: NGS Analysis Pipeline

Easy-to-follow workflow for processing Illumina whole-genome resequencing reads for population genomics. Steps include trimming, mapping, sorting, variant calling, variant filtering, PCA, and ADMIXTURE analysis. For more details, see the bottom of this notebook and visit the GitHub repository.
Repository link: [Genomics Learning Workshop](https://github.com/PoODL-CES/Genomics_learning_workshop)

This workshop introduces:
- Handling raw FASTQ files
- Performing quality control and trimming
- Mapping reads to a reference genome
- Calling and filtering variants
- Running PCA and ADMIXTURE for population analysis


In [ ]:
#@title Install system-level NGS bioinformatics tools
%%time

# Install samtools, fastqc, vcftools using apt-get (stable)
!apt-get update -qq
!apt-get install -y samtools fastqc vcftools

# Install cutadapt (requirement for Trim Galore, also installed via conda later)
!pip install --quiet cutadapt

In [ ]:
# Miniconda installation and environment setup for Colab NGS Workshop

# Download and install Miniconda (skip if already installed)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /usr/local/miniconda

import sys, os
sys.path.append('/usr/local/miniconda/lib/python3.8/site-packages')
os.environ['PATH'] = "/usr/local/miniconda/bin:" + os.environ['PATH']

# Explicitly clear potentially problematic environment variables from Python's os.environ
if 'CONDA_PREFIX' in os.environ:
    del os.environ['CONDA_PREFIX']
if 'CONDA_ENVS_PATH' in os.environ:
    del os.environ['CONDA_ENVS_PATH']

# Accept ToS for main and R conda channels
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Create new conda environment for your workshop if it doesn't exist
!if ! conda info --envs | grep -w 'workshop_ngs'; then \
    conda create -y -n workshop_ngs python=3.7; \
else \
    echo "Environment 'workshop_ngs' already exists, skipping creation."; \
fi

# Install necessary bioinformatics tools into the environment
!conda install -y -n workshop_ngs -c bioconda -c conda-forge trim-galore samtools fastqc vcftools gatk4

# Verify tool versions inside activated environment in one shell session
!bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && \
          trim_galore --version && samtools --version && fastqc --version && \
          vcftools --version && gatk --version"

To activate the `workshop_ngs` environment for specific commands and ensure its tools are used, you can wrap your commands within a `bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && your_command_here"` block. This ensures the environment is properly sourced before the command runs. Let's try listing the environment contents and checking `CONDA_PREFIX` within that activated session:

In [ ]:
!bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && \
          echo '--- Environment activated ---' && \
          echo 'CONDA_PREFIX in activated env: '$CONDA_PREFIX && \
          echo '--- Listing contents of activated env ---' && \
          conda list"